In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

ModuleNotFoundError: No module named 'pandas'

In [2]:
df = pd.read_excel("dataset/updated_text_data.xlsx")


In [3]:
df_1=df[df['गुनासो वर्ग']=='वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी ']
df_2=df[df['गुनासो वर्ग']=='सोधपुछ, सुझाव, प्रशंसा सम्बन्धी'].sample(2863,replace=True)
df_2 = df_2.reset_index(drop=True)
df_3=df[df['गुनासो वर्ग']=='कर्मचारी सम्वन्धी '].sample(2863,replace=True)
df_3 = df_3.reset_index(drop=True)
df_4=df[df['गुनासो वर्ग']=='स्वास्थ्यसँग सम्बन्धी'].sample(2863,replace=True)
df_4 = df_4.reset_index(drop=True)
df_5=df[df['गुनासो वर्ग']=='अर्थ सबन्धी '].sample(2863,replace=True)
df_5 = df_5.reset_index(drop=True)
df_6=df[df['गुनासो वर्ग']=='खानेपानी सम्बन्धी '].sample(2863,replace=True)
df_6 = df_6.reset_index(drop=True)
df_7=df[df['गुनासो वर्ग']=='सूचना तथा  संचार सम्बन्धी'].sample(2863,replace=True)
df_7 = df_7.reset_index(drop=True)
df_8=df[df['गुनासो वर्ग']=='शान्ति सुरक्षा सम्बन्धी '].sample(2863,replace=True)
df_8 = df_8.reset_index(drop=True)
df_9=df[df['गुनासो वर्ग']=='प्राकृतिक श्रोत/साधन सम्बन्धी'].sample(2863,replace=True)
df_9 =df_9.reset_index(drop=True)
df_10=df[df['गुनासो वर्ग']=='लागु पदार्थ सम्बन्धी '].sample(2863,replace=True)
df_10 = df_10.reset_index(drop=True)
df_11=df[df['गुनासो वर्ग']=='अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी '].sample(2863,replace=True)
df_11 = df_11.reset_index(drop=True)



In [4]:
df_final=pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11],axis=0)

In [5]:
# Data Preprocessing
nepali_stopwords = set(stopwords.words('nepali'))
english_stopwords = set(stopwords.words('english'))
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[#\\/।(),०-९<<?!,—–’‘:\u200d]', '', text)
    text = text.strip('"')

    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in nepali_stopwords and word.lower() not in english_stopwords]
    processed_text = ' '.join(filtered_words)
    return processed_text

df_final['गुनासो'] = df_final['गुनासो'].apply(preprocess_text)

In [6]:
# Encoding Labels
le = LabelEncoder()
df_final['label_encoded'] = le.fit_transform(df_final['गुनासो वर्ग'])


In [7]:
# Split the dataset into training and testing sets
train_data, test_data = train_test_split(df_final, test_size=0.2, stratify=df_final['label_encoded'], random_state=42)


In [8]:
# Tokenize the text data
max_words = 10000
max_length = 100
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')

tokenizer.fit_on_texts(train_data['गुनासो'])

In [9]:
train_sequences = tokenizer.texts_to_sequences(train_data['गुनासो'])

test_sequences = tokenizer.texts_to_sequences(test_data['गुनासो'])

In [10]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

In [11]:
# Load your pre-trained Nepali Word2Vec model
word2vec_model_path = "nepaliW2V_5Million.model"
word2vec_model = Word2Vec.load(word2vec_model_path)

In [12]:
# Create an embedding matrix
embedding_dim = word2vec_model.vector_size
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words and word in word2vec_model.wv:
        
        embedding_matrix[i] = word2vec_model.wv[word]

In [14]:
# Build the Bi-LSTM model with Word2Vec embeddings
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(units=100, return_sequences=True)))
model.add(Bidirectional(LSTM(units=50)))
model.add(Dense(units=len(np.unique(df_final['label_encoded'])), activation='softmax'))

In [15]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 20
model.fit(train_padded, train_data['label_encoded'], epochs=epochs, validation_data=(test_padded, test_data['label_encoded']))


Epoch 1/20


2023-12-10 21:34:06.160777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2023-12-10 21:34:07.485570: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f778783d8d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-10 21:34:07.485631: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-12-10 21:34:07.499683: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-10 21:34:07.663006: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


788/788 [==============================] - 25s 24ms/step - loss: 1.1522 - accuracy: 0.6127 - val_loss: 0.9171 - val_accuracy: 0.6771
Epoch 2/20
788/788 [==============================] - 19s 24ms/step - loss: 0.8008 - accuracy: 0.7167 - val_loss: 0.7855 - val_accuracy: 0.7236
Epoch 3/20
788/788 [==============================] - 18s 23ms/step - loss: 0.6866 - accuracy: 0.7566 - val_loss: 0.7106 - val_accuracy: 0.7473
Epoch 4/20
788/788 [==============================] - 18s 23ms/step - loss: 0.6288 - accuracy: 0.7757 - val_loss: 0.6717 - val_accuracy: 0.7644
Epoch 5/20
788/788 [==============================] - 18s 23ms/step - loss: 0.5704 - accuracy: 0.7947 - val_loss: 0.6453 - val_accuracy: 0.7782
Epoch 6/20
788/788 [==============================] - 19s 24ms/step - loss: 0.5375 - accuracy: 0.8020 - val_loss: 0.6633 - val_accuracy: 0.7739
Epoch 7/20
788/788 [==============================] - 18s 23ms/step - loss: 0.5280 - accuracy: 0.8084 - val_loss: 0.6354 - val_accuracy: 0.7843
Epo

In [16]:
# Evaluate the model
y_pred_probs = model.predict(test_padded)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# Decode the encoded labels back to original classes
y_true = le.inverse_transform(test_data['label_encoded'])
y_pred_decoded = le.inverse_transform(y_pred)

# Print the classification report
from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(y_true, y_pred_decoded))


197/197 [==============================] - 2s 7ms/step
Classification Report:
                                           precision    recall  f1-score   support

                             अर्थ सबन्धी        0.91      0.79      0.85       572
अर्थिक अनियमितता तथा भ्रष्टाचार सम्बन्धी        0.99      1.00      1.00       573
                       कर्मचारी सम्वन्धी        0.91      0.83      0.87       573
                       खानेपानी सम्बन्धी        0.94      0.77      0.84       573
            प्राकृतिक श्रोत/साधन सम्बन्धी       0.97      0.88      0.92       573
                    लागु पदार्थ सम्बन्धी        1.00      0.97      0.98       572
  वेबसाइट तथा अभिलेख व्यवस्थापन सम्बन्धी        0.33      0.76      0.46       573
                 शान्ति सुरक्षा सम्बन्धी        0.91      0.60      0.72       573
                सूचना तथा  संचार सम्बन्धी       0.93      0.85      0.89       573
          सोधपुछ, सुझाव, प्रशंसा सम्बन्धी       0.81      0.67      0.74       572
        

In [17]:
# New Nepali text
new_text = "यस महानगरपालिकाको सुनाकोठी नखिपोट नख्खुडोल गोकुल आवास कान्तिपुर कोलोनीका वासिन्दाले दिनरात यो धुंवाको सास कहिलेसम्म फेर्नुपर्ने हो हजुर?"

# Preprocess the new text
processed_text = preprocess_text(new_text)

# Tokenize and pad the sequence
new_sequence = tokenizer.texts_to_sequences([processed_text])
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Make predictions
new_pred_probs = model.predict(new_padded)
new_pred_class = np.argmax(new_pred_probs, axis=1)[0]
predicted_class_label = le.inverse_transform([new_pred_class])[0]

# Print the result
print(f"Predicted Class: {predicted_class_label} (Class {new_pred_class})")

1/1 [==============================] - 0s 31ms/step
Predicted Class: कर्मचारी सम्वन्धी  (Class 2)


In [18]:
# New Nepali text
new_text = "सुनसरी जिल्ला इटहरी उप महानगर भित्र अवस्थित धरान पुग्ने यो बाटो । कुन ठेकेदारले यो काम गर्दा कति पैसा कुम्ल्याउन पाइने हो र आफ्नो निर्वाचन ताका खर्च भएको करोड उठाउन पाइन्छ भनेर त हैन ?"

# Preprocess the new text
processed_text = preprocess_text(new_text)

# Tokenize and pad the sequence
new_sequence = tokenizer.texts_to_sequences([processed_text])
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Make predictions
new_pred_probs = model.predict(new_padded)
new_pred_class = np.argmax(new_pred_probs, axis=1)[0]
predicted_class_label = le.inverse_transform([new_pred_class])[0]

# Print the result
print(f"Predicted Class: {predicted_class_label} (Class {new_pred_class})")

1/1 [==============================] - 0s 23ms/step
Predicted Class: सोधपुछ, सुझाव, प्रशंसा सम्बन्धी (Class 9)


In [19]:
# New Nepali text
new_text = "इटहरीदेखि पूर्वाञ्चल विश्वविद्यालय, माेरङ हुँदै पूर्व जाने बाटाे जुन लालभित्तीमा गएर टुङ्गिन्छ, पिच गर्नुपर्‍याे। बीचमा दुइटा ठूला खाेला छन्, त्यसमा पुल हाल्नुपर्‍याे। याे बाटाेलाई वैकल्पिक राजमार्गकाे रुपमा विकास गर्नुप्रयो।"

# Preprocess the new text
processed_text = preprocess_text(new_text)

# Tokenize and pad the sequence
new_sequence = tokenizer.texts_to_sequences([processed_text])
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Make predictions
new_pred_probs = model.predict(new_padded)
new_pred_class = np.argmax(new_pred_probs, axis=1)[0]
predicted_class_label = le.inverse_transform([new_pred_class])[0]

# Print the result
print(f"Predicted Class: {predicted_class_label} (Class {new_pred_class})")

1/1 [==============================] - 0s 20ms/step
Predicted Class: सोधपुछ, सुझाव, प्रशंसा सम्बन्धी (Class 9)


In [20]:
# New Nepali text
new_text = "मेयरले उपभोक्ता समितिसँग ३० प्रतिशत कमिसन लिएपछि विकासको काम गुणस्तरहीन, दबाब झेल्न नसकेर प्रशासकीय अधिकृतको भागाभाग। बाह्रबिसेका मेयरको मनोमानी- दोहोरीमा रमाइलो गरेको बिलसमेत नगरपालिकाबाटै भुक्तानी गर्न दबाब मेयरले उपभोक्ता समितिसँग ३० प्रतिशत कमिसन लिएपछि विकासको काम गुणस्तरहीन, दबाब झेल्न नसकेर प्रशासकीय अधिकृतको भागाभाग।"

# Preprocess the new text
processed_text = preprocess_text(new_text)

# Tokenize and pad the sequence
new_sequence = tokenizer.texts_to_sequences([processed_text])
new_padded = pad_sequences(new_sequence, maxlen=max_length, padding='post', truncating='post')

# Make predictions
new_pred_probs = model.predict(new_padded)
new_pred_class = np.argmax(new_pred_probs, axis=1)[0]
predicted_class_label = le.inverse_transform([new_pred_class])[0]

# Print the result
print(f"Predicted Class: {predicted_class_label} (Class {new_pred_class})")

1/1 [==============================] - 0s 23ms/step
Predicted Class: अर्थ सबन्धी  (Class 0)
